# 3. Naiver Bayesklassifikator zur Gesichtserkennung

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
%load_ext version_information

Implementieren Sie den Gaussian-Naïve-Bayes-Klassifikator aus der Vorlesung.

In [ ]:
from scipy.stats import norm

class GaussianNaiveBayes:
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.mean = {}
        self.std = {}
        self.class_prior = {}

        # Calculate mean and standard deviation for each feature in each class
        for c in self.classes:
            X_c = X[y == c]
            self.mean[c] = np.mean(X_c, axis=0)
            self.std[c] = np.std(X_c, axis=0)

            # Calculate class prior probability
            self.class_prior[c] = len(X_c) / len(X)

    def predict(self, X):
        predictions = []

        # For every feature
        for x in X:
            posteriors = []

            # Calculate posterior probability for each class
            for c in self.classes:
                prior = np.log(self.class_prior[c])
                likelihood = np.sum(np.log(norm.pdf(x, loc=self.mean[c], scale=self.std[c])))
                posterior = prior + likelihood
                posteriors.append(posterior)

            # Append the class with the highest posterior probability to predictions
            predictions.append(self.classes[np.argmax(posteriors)])

        return np.array(predictions)

Testen Sie Ihre Implementierung am Datensatz ''Labeled Faces in the Wild'' aus Aufgabe 2, wiederum nur für Personen, für die mindestens 70 Bilder existieren.

In [ ]:
import os
import tarfile
from urllib.request import urlretrieve

lfw_filename = 'lfw-funneled.tgz'
lfw_directory = '.lfw-dataset'

if not os.path.isfile(lfw_filename):
    print("Downloading")
    urlretrieve('http://vis-www.cs.umass.edu/lfw/lfw-funneled.tgz',filename = lfw_filename)

if not os.path.isdir(lfw_directory):
    # Dateien in das Zielverzeichnis extrahieren
    with tarfile.open(lfw_filename, 'r:gz') as tar:
        tar.extractall(path=lfw_directory)

min_images_required = 70
selected_persons = []
extracted_path = ".lfw-dataset/lfw_funneled"

# Verzeichnis durchsuchen
for person_folder in os.listdir(extracted_path):
    person_path = os.path.join(extracted_path, person_folder)

    if os.path.isdir(person_path):
        # Anzahl der Bilder fuer die aktuelle Person zaehlen
        num_images = len([f for f in os.listdir(person_path) if f.endswith('.jpg')])

        # Ueberpruefen, ob Mindestanzahl erreicht
        if num_images >= min_images_required:
            selected_persons.append({
                'person_name': person_folder,
                'num_images': num_images
            })

print(f"Personen mit mindestens {min_images_required} Bildern:")
for person_info in selected_persons:
    print(f"{person_info['person_name']}: {person_info['num_images']} Bilder")

Teilen Sie Ihren Datensatz in 60 % Trainings- und 40% Testdaten (nach vorheriger Zufalls-Permutation der Reihenfolge) und skalieren Sie die Bilder wieder auf 1/8 der Originalgröße.

In [ ]:
from skimage import io, transform
data = []
names = []

# Daten verarbeiten
for person in selected_persons:
    person_folder = person['person_name']
    person_path = os.path.join(extracted_path, person_folder)
    all_person_data = []

    # Bilder laden
    for filename in os.listdir(person_path):
        if filename.endswith('.jpg'):
            image_path = os.path.join(person_path, filename)
            # Laden und in Graustufen konvertieren
            img = io.imread(image_path, as_gray=True)
            # Skalieren auf 32x32
            img = transform.resize(img, (32, 32))
            # In einen Vektor packen
            img_vector = img.flatten()
            data.append(img_vector)
            names.append(person_folder)

# Kombiniere die beiden Listen zu einer Liste von Tupeln
data_and_names = list(zip(data, names))

# Zufalls-Permutation der Reihenfolge
import random
random.shuffle(data_and_names)

# Teile die kombinierten und zufällig angeordneten Listen in Trainings- und Testlisten auf (60% Train, 40% Test)
from sklearn.model_selection import train_test_split
train_tuples, test_tuples = train_test_split(data_and_names, test_size=0.4, random_state=42)

# Teile die Trainings- und Testlisten wieder in die ursprünglichen Listen auf
train_data, train_names = zip(*train_tuples)
train_data = np.array(train_data)
test_data, test_names = zip(*test_tuples)
test_data = np.array(test_data)

 Führen Sie anschließend eine Hauptkomponentenanalyse auf den Trainingsdaten durch und projizieren Sie sowohl Trainings- als auch Testbilder auf die ersten 7 Eigengesichter.

In [ ]:
X = pd.DataFrame(train_data)

from pca import pca
num_components = len(X.columns)
X_pca, Sigma, V = pca(X, num_components)

# Umrechnung von Singulärwerten in Eigenwerte
eigenvalues = (Sigma**2) / (len(X) - 1)

# Gesamtvarianz
total_variance = sum(eigenvalues)

# Erklärte Varianz für jede Komponente
explained_variances = [(i / total_variance) * 100 for i in eigenvalues]

# Kumulative erklärte Varianz
cumulative_variances = np.cumsum(explained_variances)

# Tabellarische Darstellung
results = pd.DataFrame({
    'Eigenwert': eigenvalues,
    'Erklärte Varianz (%)': explained_variances,
    'Kumulative erklärte Varianz (%)': cumulative_variances
})

print("Ergebnisse der PCA-Analyse:")
print(results.head(150))

# Trainingsdaten projeziert auf die ersten 7 Eigengesichter
train_data_pca = X_pca[:,:7]

# Testdaten zentrieren
test_data_meaned = pd.DataFrame(test_data) - np.mean(X, axis=0)

# Testdaten standardisieren
test_data_std = np.std(test_data_meaned, axis=0)
test_data_standardized = test_data_meaned / test_data_std

# Projektion der Testdaten auf die ersten 7 Eigengesichter
test_data_pca = np.dot(test_data_standardized, V[:,:7])

 Trainieren Sie Ihren GNB-Klassifikator auf dem Trainingsdatensatz als ''George-W.-Bush-Detektor'', d.h. alle zu dieser Person gehörigen Bilder werden mit 1 gelabelt, alle sonstigen mit –1. Werten Sie Ihren Klassifikator sowohl auf den Trainings- wie auf den unabhängigen Testdaten aus. Bestimmen Sie dafür jeweils die Detektionswahrscheinlichkeit, Richtig-Negativ-Rate, Fehlalarmrate und Falsch-Negativ-Rate.

In [ ]:
train_labels = np.array([1 if name == 'George_W_Bush' else -1 for name in train_names])
test_labels = np.array([1 if name == 'George_W_Bush' else -1 for name in test_names])

model = GaussianNaiveBayes()
model.fit(train_data, train_labels)

# Vorhersagen für Trainingsdaten
train_pred = model.predict(train_data)

# True Positives, True Negatives, False Positives und False Negatives
tp = np.sum((train_labels == 1) & (train_pred == 1))
tn = np.sum((train_labels == -1) & (train_pred == -1))
fp = np.sum((train_labels == -1) & (train_pred == 1))
fn = np.sum((train_labels == 1) & (train_pred == -1))

# Berechnung der Detektionswahrscheinlichkeit, Richtig-Negativ-Rate, Fehlalarmrate und Falsch-Negativ-Rate
detection_probability = tp / (tp + fn)
true_negative_rate = tn / (tn + fp)
false_alarm_rate = 1 - true_negative_rate
false_negative_rate = 1 - detection_probability

print("---Trainingsdaten---")
print(f"Detektionswahrscheinlichkeit: {detection_probability}")
print(f"Richtig-Negativ-Rate: {true_negative_rate}")
print(f"Fehlalarmrate: {false_alarm_rate}")
print(f"Falsch-Negativ-Rate: {false_negative_rate}")

In [ ]:
# Vorhersagen für Testdaten
test_pred = model.predict(test_data)

# True Positives, True Negatives, False Positives und False Negatives
tp = np.sum((test_labels == 1) & (test_pred == 1))
tn = np.sum((test_labels == -1) & (test_pred == -1))
fp = np.sum((test_labels == -1) & (test_pred == 1))
fn = np.sum((test_labels == 1) & (test_pred == -1))

# Berechnung der Detektionswahrscheinlichkeit, Richtig-Negativ-Rate, Fehlalarmrate und Falsch-Negativ-Rate
detection_probability = tp / (tp + fn)
true_negative_rate = tn / (tn + fp)
false_alarm_rate = 1 - true_negative_rate
false_negative_rate = 1 - detection_probability

print("---Testdaten---")
print(f"Detektionswahrscheinlichkeit: {detection_probability}")
print(f"Richtig-Negativ-Rate: {true_negative_rate}")
print(f"Fehlalarmrate: {false_alarm_rate}")
print(f"Falsch-Negativ-Rate: {false_negative_rate}")

In [ ]:
%version_information